K-means clustering
===========

The k-means clustering algorithm is an unsupervised learning method broadly used in cluster analysis. the algorithm is based on two main steps:

1. **Assignment**: each observation in the input space is assigned to the Best Matching Unit (BMU) based on a distance measure.
2. **Update**: for each cluster estimate the mean and assign it to the centroids.

In general the distance measure used is th [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance). A more efficient version is the [cosine distance](https://en.wikipedia.org/wiki/Cosine_similarity) that is based on the inner product. The goal in k-means clustering is to minimize the within-cluster sum of squares (reconstruction error):

$$\underset{S}{\mathrm{argmin}} \sum_{i=1}^{k} \sum_{\bf{x} \in S_{i}} \lVert \bf{x} - \bf{\mu}_{i} \rVert^{2}$$

where $\bf{x}$ are the input vectors, $S = \{ S_{1},...,S_{k} \}$ are the number of clusters, and $\mu$ is the mean of the vectors belonging to each cluster.

Online k-means algorithm in Tensorflow
--------------------------------------------

Here I show you how to create an online version of the k-means algorithm. By *online* I mean that a batch of examples can be passed every time the training operation is called. This is particularly useful if the input space is particularly big. First of all we declare the input size and the number of clusters we want to use:

In [ ]:
input_size = 6
number_centroids = 2 #the number of centroids
batch_size = 3 #The number of input vector to pass

Now we can write the real algortihm. The centroids and the input are assigned through two placeholders. The distance used is the **Euclidean distance**. Remember that we want to minimize the within-cluster sum of squares (reconstruction error) between the input vectors and the centroids. Since the problem can be defined in term of a loss function, we can use our dear optimizers.

In [ ]:
import tensorflow as tf

#Placeholders for the input array and the initial centroids
input_placeholder = tf.placeholder(dtype=tf.float32, shape=[None, input_size])
initial_centroids_placeholder = tf.placeholder(dtype=tf.float32, shape=[input_size, number_centroids])

#Matrix containing the centorids
kmeans_matrix = tf.Variable(tf.random_uniform(shape=[input_size, number_centroids], 
                                              minval=0.0, maxval=1.0, dtype=tf.float32))
assign_centroids_op = kmeans_matrix.assign(initial_centroids_placeholder)

#Here the distance is estimated and the BMU computed
difference = tf.expand_dims(input_placeholder, axis=1) - tf.expand_dims(tf.transpose(kmeans_matrix), axis=0)
euclidean_distance = tf.norm(difference, ord='euclidean',axis=2) #shape=(?, 3)
bmu_index = tf.argmin(euclidean_distance, axis=1) #get the index of BMU
bmu = tf.gather(kmeans_matrix, indices=bmu_index, axis=1) #take the centroinds

#To minimize: within-cluster sum of squares (reconstruction error)
loss = tf.reduce_mean(tf.pow(input_placeholder - tf.transpose(bmu), 2))

#optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
optimizer = tf.train.AdamOptimizer(learning_rate=0.05)

#Training operation
train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())

Now the graph is ready and we can start a session to try the algorithm. Here I simply create a certain number of random input arrays. The initialization of the centroids is trivial, I take some of the input arrays and I pass them to the centroids.

In [ ]:
import numpy as np

sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_array = np.random.random((batch_size,input_size))
print("\ninput_array.T")
print(input_array.T)
print("\nkmeans_matrix before assignment")
print(sess.run([kmeans_matrix]))
sess.run([assign_centroids_op], 
         {initial_centroids_placeholder: input_array[0:number_centroids,:].T})
print("\nkmeans_matrix after assignment")
print(sess.run([kmeans_matrix]))

print("\nStarting training...")
print_every = 100
for i in range(1000):
    output = sess.run([train_op, loss], {input_placeholder: input_array})
    if(i%print_every==0):
        print("Loss: " + str(output[1]))

print("\nkmeans_matrix final")
print(sess.run([kmeans_matrix]))

Training on the Iris dataset
--------------------------------

Now we can try the algorithm on a real dataset. We can use the [Iris Flower dataset](../iris/iris.ipynb) for instance. The dataset is represented by the dimensions of the flowers: sepal length, sepal width, petal length, petal width. There are a total of three classes (0=Setosa, 1=Versicolor, 2=Virginica).
The TFRecord files for this dataset are already included in Tensorbag and we can load them in memory with the following snippet:

In [ ]:
import tensorflow as tf
import numpy as np

np_dataset = np.loadtxt(open("../iris/iris_dataset.csv", "rb"), delimiter=",", skiprows=1)
np.random.shuffle(np_dataset) #shuffle the dataset rows
train_features_array = np_dataset[0:100, 0:4]
train_labels_array = np_dataset[0:100, 4:].flatten().astype(np.int32)
test_features_array = np_dataset[100:150, 0:4]
test_labels_array = np_dataset[100:150, 4:].flatten().astype(np.int32)

print(train_features_array.shape)
print(train_labels_array.shape)
print(test_features_array.shape)
print(test_labels_array.shape)

In [ ]:
input_size = 4 #sepal length, sepal width, petal length, petal width
number_centroids = 3 #the number of centroids is equal to the number of the classes

#Placeholders for the input array and the initial centroids
input_placeholder = tf.placeholder(dtype=tf.float32, shape=[None, input_size])
initial_centroids_placeholder = tf.placeholder(dtype=tf.float32, shape=[input_size, number_centroids])

#Matrix containing the centorids
kmeans_matrix = tf.Variable(tf.random_uniform(shape=[input_size, number_centroids], 
                                              minval=0.0, maxval=1.0, dtype=tf.float32))
assign_centroids_op = kmeans_matrix.assign(initial_centroids_placeholder)

#Here the distance is estimated and the BMU computed
difference = tf.expand_dims(input_placeholder, axis=1) - tf.expand_dims(tf.transpose(kmeans_matrix), axis=0)
euclidean_distance = tf.norm(difference, ord='euclidean',axis=2) #shape=(?, 3)
bmu_index = tf.argmin(euclidean_distance, axis=1) #get the index of BMU
bmu = tf.gather(kmeans_matrix, indices=bmu_index, axis=1) #take the centroids

#To minimize: within-cluster sum of squares (reconstruction error)
loss = tf.reduce_mean(tf.pow(input_placeholder - tf.transpose(bmu), 2))

#optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
optimizer = tf.train.AdamOptimizer(learning_rate=0.05)
#optimizer = tf.train.RMSPropOptimizer(learning_rate=0.1)

#Training operation
train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())

In [ ]:
tot_epochs = 1000

sess = tf.Session()
sess.run(tf.global_variables_initializer())
print("\nkmeans_matrix before assignment")
print(sess.run([kmeans_matrix]))
input_array = train_features_array
sess.run([assign_centroids_op], 
         {initial_centroids_placeholder: input_array[0:number_centroids,:].T})
print("\nkmeans_matrix after assignment")
print(sess.run([kmeans_matrix]))

print("\nLoss before training")
output = sess.run([loss], {input_placeholder: input_array})
print(output)

print("\nStarting training...")

for iteration in range(tot_epochs):
        output = sess.run([train_op, loss], {input_placeholder: input_array})
        print("Loss: " + str(output[1]))
        
print("\nkmeans_matrix final")
print(sess.run([kmeans_matrix]))

Testing the algorithm
-------------------------

After the training we can test the results. For the testing phase I assign each element to the closest centroid and I evaluate the **purity**. the purity is defined as the sum of the elements in each cluster that belong to the most numerous class, divided by the total number of elements. For instance, in the Iris dataset a purity of 1.0 can be obtained if the elements belonging to the three different classes are confined in three different clusters. It is possible to have purity one in the edge case of $K \geq N$ meaning that the number of clusters are more than the number of elements.

In [ ]:
#Get the indices of the test array
indices = sess.run([bmu_index], {input_placeholder: test_features_array})[0]

print("\nTest classes:")
print(test_labels_array)
print("\nBMU indices:")
print indices

cluster_class_matrix = np.zeros((number_centroids,3))
for i in range(len(indices)):
    cluster = indices[i]
    label = int(test_labels_array[i])
    cluster_class_matrix[cluster,label] += 1
purity = np.sum(np.max(cluster_class_matrix, axis=1)) / np.sum(cluster_class_matrix)

print("\nCluster-class matrix:")
print cluster_class_matrix

print("\nPurity: " + str(purity))

Using Adam or RMSProp optimizer it is possible to obtain purity of 0.8 using three clusters. I invite you to play with the number of clusters and the optimizer to see how the purity change.

**Copyright (c) 2018** Massimiliano Patacchiola, MIT License